In [1]:
import scipy.io.wavfile as wav
import os
import numpy as np
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import librosa
import IPython.display as ipd

In [2]:
def normalize_array_to_minmax(one_darray,amp):
    """normalize values to x/(xmaxormin) * amp and"""
    
    max_val = np.max(one_darray)
    min_val = np.min(one_darray)
    
    
    for element in one_darray:
        if(element > 0):
            element = int(float(element)/float(max_val) * float(amp))
        if(element < 0):
            element = -1*(int(float(element)/float(min_val) * float(amp)))

    return one_darray
        
        

def find_index_of_val(one_darray,value):
    index = 0
    
    for element in one_darray:
        if element == value:
            return index
        index +=1
    
def find_center(one_darray,highpoint,lowpoint):
    return int(np.mean([find_index_of_val(one_darray,highpoint),find_index_of_val(one_darray, lowpoint)]))
    

def cut_and_normalize_array(one_darray, highint=8000):
    """ cut before and after the highest/ lowest peak   """
    max_val = np.max(one_darray)
    min_val = np.min(one_darray)
        
    # buffer the array with 1500 upstream and 2500 downstream noise
    
    one_darray = np.hstack((np.zeros(2000), one_darray,np.zeros(3000)))
    
    center = find_center(one_darray,max_val,min_val)
    
    one_darray = normalize_minmax_vectorized(one_darray,highint,max_val,min_val)
    # hardcode the array
    
    return one_darray[(center-2000):(center+3000)]
    
    #print(center,"  ",normalize_minmax_vectorized(one_darray,highint,max_val,min_val))
    

In [10]:
audiofilepath = '../data/train/audio/cat/004ae714_nohash_0.wav'
rate,data = wav.read(audiofilepath)
center = find_center(data,np.max(data),np.min(data))

silence = data[:center-2000]

while len(silence) < 16000:
    silence = np.vstack((silence,silence))

print(len(silence[:16000]))


16000


In [4]:
#moddir = '../data/mod/train/audio/'
train_folder = '../data/train/audio/'

files_written = 0

for (dirpath, dirnames, filenames) in os.walk(train_folder):
    #if count < maxcount:
        for dirs in dirnames:
            for (dirpath, dirnames, filenames) in os.walk('../data/train/audio/' + str(dirs)):
                for files in filenames:
                
                    if files.endswith('.wav'):
                        #try:
                        
                            audiofilepath = '%s%s/%s' %(train_folder,dirs,files)
                            rate,data = wav.read(audiofilepath)
                            
                            center = find_center(data,np.max(data),np.min(data))
                            if center > 10000:
                                
                                files_written +=1
                                
                                silence = data[:center-4000]
                            
                                while len(silence) < 16000:
                                    silence = np.hstack((silence,silence))


                                wav.write(filename='../data/train/audio/silence/%s_silence.wav' %(files), 
                                          data=np.ndarray.astype(silence[:16000],np.int16), rate=16000)

                                
print("wrote " ,files_written, " files")

KeyboardInterrupt: 